# 3. Keras Model

Create a simple Deep Learning Model using Keras

In [1]:
import pandas as pd
from keras import models
from keras import layers

2022-12-31 11:58:37.210120: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-31 11:58:37.857798: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-31 11:58:37.857842: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-31 11:58:37.857848: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

## 3.1. Data Collection

In [2]:
# load the source data
df_source = pd.read_csv('data/raw/DataCoSupplyChainDataset.csv', encoding='unicode_escape')
df_source.drop_duplicates(inplace=True)

print(df_source.shape)
with pd.option_context('display.max_columns', None):
    display(df_source.head())

(180519, 53)


,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,Customer Country,Customer Email,Customer Fname,Customer Id,Customer Lname,Customer Password,Customer Segment,Customer State,Customer Street,Customer Zipcode,Department Id,Department Name,Latitude,Longitude,Market,Order City,Order Country,Order Customer Id,order date (DateOrders),Order Id,Order Item Cardprod Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Cally,20755,Holloway,XXXXXXXXX,Consumer,PR,5365 Noble Nectar Island,725.0,2,Fitness,18.251453,-66.037056,Pacific Asia,Bekasi,Indonesia,20755,1/31/2018 22:56,77202,1360,13.110000,0.04,180517,327.75,0.29,1,327.75,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Irene,19492,Luna,XXXXXXXXX,Consumer,PR,2679 Rustic Loop,725.0,2,Fitness,18.279451,-66.037064,Pacific Asia,Bikaner,India,19492,1/13/2018 12:27,75939,1360,16.389999,0.05,179254,327.75,-0.80,1,327.75,311.359985,-249.089996,South Asia,Rajastán,PENDING,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,EE. UU.,XXXXXXXXX,Gillian,19491,Maldonado,XXXXXXXXX,Consumer,CA,8510 Round Bear Gate,95125.0,2,Fitness,37.292233,-121.881279,Pacific Asia,Bikaner,India,19491,1/13/2018 12:06,75938,1360,18.030001,0.06,179253,327.75,-0.80,1,327.75,309.720001,-247.779999,South Asia,Rajastán,CLOSED,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,EE. UU.,XXXXXXXXX,Tana,19490,Tate,XXXXXXXXX,Home Office,CA,3200 Amber Bend,90027.0,2,Fitness,34.125946,-118.291016,Pacific Asia,Townsville,Australia,19490,1/13/2018 11:45,75937,1360,22.940001,0.07,179252,327.75,0.08,1,327.75,304.809998,22.860001,Oceania,Queensland,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Orli,19489,Hendricks,XXXXXXXXX,Corporate,PR,8671 Iron Anchor Corners,725.0,2,Fitness,18.253769,-66.037048,Pacific Asia,Townsville,Australia,19489,1/13/2018 11:24,75936,1360,29.500000,0.09,179251,327.75,0.45,1,327.75,298.250000,134.210007,Oceania,Queensland,PENDING_PAYMENT,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


## 3.2 Data Preparation & Cleaning

### 3.2.1 Add the Target Variable

In [3]:
def add_is_fraud(df_data: pd.DataFrame) -> pd.DataFrame:
    """
    Add a new column to the dataframe that indicates
    0: No Fraud and 1: Fraud

    Parameters
    ----------
    df_data : pd.DataFrame
        The source dataframe
    """
    df_data['is_fraud'] = df_data['Order Status'].apply(lambda x: 1 if x == 'SUSPECTED_FRAUD' else 0)
    return df_data

df_data = df_source.reset_index(drop=True)
add_is_fraud(df_data)
df_data['is_fraud'].value_counts()

0    176457
1      4062
Name: is_fraud, dtype: int64

### 3.2.2 Data Selection

Select the subset of columns to use for training the model.

In [4]:
df_data = df_data \
    .filter([
        'is_fraud',
        'Customer Id',
        'Order Id',
        'Type',
        'Customer Country',
        'Customer State',
        'Customer City',
        'Order Country',
        'order date (DateOrders)',
        'Sales per customer'
    ])

print(df_data.shape)
with pd.option_context('display.max_columns', None):
    display(df_data.head())

(180519, 10)


,is_fraud,Customer Id,Order Id,Type,Customer Country,Customer State,Customer City,Order Country,order date (DateOrders),Sales per customer
0,0,20755,77202,DEBIT,Puerto Rico,PR,Caguas,Indonesia,1/31/2018 22:56,314.640015
1,0,19492,75939,TRANSFER,Puerto Rico,PR,Caguas,India,1/13/2018 12:27,311.359985
2,0,19491,75938,CASH,EE. UU.,CA,San Jose,India,1/13/2018 12:06,309.720001
3,0,19490,75937,DEBIT,EE. UU.,CA,Los Angeles,Australia,1/13/2018 11:45,304.809998
4,0,19489,75936,PAYMENT,Puerto Rico,PR,Caguas,Australia,1/13/2018 11:24,298.250000


### 3.2.3. Summarize Data

In [5]:
# group the data and get the order total
df_order_total = df_data \
    .groupby(['Customer Id', 'Order Id'], as_index=False) \
    .agg({'Sales per customer': 'sum'}) \
    .rename(columns={'Sales per customer': 'order_total'}) \
    .sort_values(by=['Customer Id', 'Order Id'], ascending=True) \
    .reset_index(drop=True)

df_order_total.head()

,Customer Id,Order Id,order_total
0,1,22945,472.450012
1,2,15192,84.970001
2,2,33865,506.970016
3,2,57963,674.740013
4,2,67863,351.980011


In [6]:
# group the data and get the cumulative sales per customer
df_order_total['sales_per_customer'] = df_order_total \
    .groupby('Customer Id') \
    ['order_total'].cumsum().sort_values(ascending=False)

df_order_total[df_order_total['Customer Id'] == 918]

,Customer Id,Order Id,order_total,sales_per_customer
4228,918,11,846.800003,846.800003
4229,918,8555,661.110016,1507.910019
4230,918,42147,309.189995,1817.100014


In [7]:
# join to the original dataset
df_data = df_data \
    .drop(columns=['Sales per customer']) \
    .merge(df_order_total, on=['Customer Id', 'Order Id'], how='left') \
    .drop_duplicates() \
    .sort_values(by=['Customer Id', 'Order Id'], ascending=True) \
    .reset_index(drop=True)

print(df_data.shape)
with pd.option_context('display.max_columns', None):
    display(df_data.head())

(65752, 11)


,is_fraud,Customer Id,Order Id,Type,Customer Country,Customer State,Customer City,Order Country,order date (DateOrders),order_total,sales_per_customer
0,0,1,22945,DEBIT,EE. UU.,TX,Brownsville,India,12/1/2015 22:18,472.450012,472.450012
1,0,2,15192,PAYMENT,EE. UU.,CO,Littleton,Alemania,8/10/2015 18:05,84.970001,84.970001
2,0,2,33865,DEBIT,EE. UU.,CO,Littleton,Estados Unidos,5/9/2016 8:04,506.970016,591.940018
3,0,2,57963,DEBIT,EE. UU.,CO,Littleton,Honduras,4/26/2017 2:40,674.740013,1266.680031
4,0,2,67863,DEBIT,EE. UU.,CO,Littleton,Italia,9/17/2017 15:04,351.980011,1618.660042


### 3.2.4 Configure Data Types

#### 3.2.4.1 Data Columns

In [8]:
# set the order data to datetime
df_data['order_date'] = pd.to_datetime(df_data['order date (DateOrders)'])
df_data.drop(columns=['order date (DateOrders)'], inplace=True)

#### 3.2.4.2 Categorical Columns

In [9]:
cat_columns = {
        'Type': 'payment_type',
        'Customer Country': 'customer_country',
        'Customer State': 'customer_state',
        'Customer City': 'customer_city',
        'Order Country': 'order_country'    
}

def convert_categorical(df_data: pd.DataFrame, cat_columns: dict) -> pd.DataFrame:
    """
    Convert the categorical columns to numeric

    Parameters
    ----------
    df_data : pd.DataFrame
        The source dataframe
    cat_columns : dict
        The dictionary of categorical columns

    Returns
    -------
    pd.DataFrame
        The dataframe with the columns converted to categorical and renamed.
    """
    df_output = df_data.copy()
    
    # convert the column to categorical
    for key, value in cat_columns.items():
        df_output[key] = df_output[key].astype('category')

    # rename the columns
    df_output.rename(columns=cat_columns, inplace=True)

    return df_output

df_data = convert_categorical(df_data, cat_columns)

In [10]:
df_data.dtypes

is_fraud                       int64
Customer Id                    int64
Order Id                       int64
payment_type                category
customer_country            category
customer_state              category
customer_city               category
order_country               category
order_total                  float64
sales_per_customer           float64
order_date            datetime64[ns]
dtype: object

In [11]:
# show a summary of the categorical columns
df_data \
    .filter([
        'payment_type',
        'customer_country',
        'customer_State',
        'customer_city',
        'order_country',
    ]) \
    .describe(
        include='all',
        datetime_is_numeric=True)

,payment_type,customer_country,customer_city,order_country
count,65752,65752,65752,65752
unique,4,2,563,164
top,DEBIT,EE. UU.,Caguas,Estados Unidos
freq,25340,40440,24365,8270


## 3.3 Feature Engineering

### 3.3.1 Remove Customer Id and Order Id

In [12]:
df_features = df_data \
    .drop(columns=[
        'Customer Id',
        'Order Id'
    ])

### 3.3.2 Add the date features

In [16]:
def add_date_features(df_data: pd.DataFrame) -> pd.DataFrame:
    """
    Add the date features

    Parameters
    ----------
    df_data : pd.DataFrame
        The source dataframe

    Returns
    -------
    pd.DataFrame
        The dataframe with the data features added.
    """
    df_output = df_data.copy()

    # add the month of the year
    df_output['order_month_of_year'] = df_output['order_date'].dt.month
    df_output['order_month_of_year'] = df_output['order_month_of_year'].astype('category')

    # add the hour
    df_output['order_hour'] = df_output['order_date'].dt.hour
    df_output['order_hour'] = df_output['order_hour'].astype('category')

    # drop the oder date
    df_output.drop(columns=['order_date'], inplace=True)

    return df_output

df_features = add_date_features(df_features)

print(df_features.shape)
with pd.option_context('display.max_columns', None):
    display(df_features.head())

(65752, 784)


,is_fraud,order_total,sales_per_customer,payment_type_CASH,payment_type_DEBIT,payment_type_PAYMENT,payment_type_TRANSFER,customer_country_EE. UU.,customer_country_Puerto Rico,customer_state_91732,customer_state_95758,customer_state_AL,customer_state_AR,customer_state_AZ,customer_state_CA,customer_state_CO,customer_state_CT,customer_state_DC,customer_state_DE,customer_state_FL,customer_state_GA,customer_state_HI,customer_state_IA,customer_state_ID,customer_state_IL,customer_state_IN,customer_state_KS,customer_state_KY,customer_state_LA,customer_state_MA,customer_state_MD,customer_state_MI,customer_state_MN,customer_state_MO,customer_state_MT,customer_state_NC,customer_state_ND,customer_state_NJ,customer_state_NM,customer_state_NV,customer_state_NY,customer_state_OH,customer_state_OK,customer_state_OR,customer_state_PA,customer_state_PR,customer_state_RI,customer_state_SC,customer_state_TN,customer_state_TX,customer_state_UT,customer_state_VA,customer_state_WA,customer_state_WI,customer_state_WV,customer_city_Aguadilla,customer_city_Alameda,customer_city_Albany,customer_city_Albuquerque,customer_city_Algonquin,customer_city_Alhambra,customer_city_Allentown,customer_city_Alpharetta,customer_city_Amarillo,customer_city_Anaheim,customer_city_Ann Arbor,customer_city_Annandale,customer_city_Annapolis,customer_city_Antioch,customer_city_Apex,customer_city_Apopka,customer_city_Arecibo,customer_city_Arlington,customer_city_Arlington Heights,customer_city_Asheboro,customer_city_Astoria,customer_city_Atlanta,customer_city_Augusta,customer_city_Aurora,customer_city_Austin,customer_city_Azusa,customer_city_Bakersfield,customer_city_Baldwin Park,customer_city_Ballwin,customer_city_Baltimore,customer_city_Bartlett,customer_city_Bay Shore,customer_city_Bayamon,customer_city_Bayonne,customer_city_Baytown,customer_city_Beaverton,customer_city_Bell Gardens,customer_city_Bellflower,customer_city_Bellingham,customer_city_Beloit,customer_city_Bend,customer_city_Bensalem,customer_city_Berwyn,customer_city_Billings,customer_city_Birmingham,customer_city_Bismarck,customer_city_Blacksburg,customer_city_Bloomfield,customer_city_Bolingbrook,customer_city_Bountiful,customer_city_Bowling Green,customer_city_Brandon,customer_city_Brentwood,customer_city_Bridgeton,customer_city_Brighton,customer_city_Bristol,customer_city_Brockton,customer_city_Broken Arrow,customer_city_Bronx,customer_city_Brooklyn,customer_city_Broomfield,customer_city_Brownsville,customer_city_Buena Park,customer_city_Buffalo,customer_city_Burnsville,customer_city_CA,customer_city_Caguas,customer_city_Campbell,customer_city_Canoga Park,customer_city_Canovanas,customer_city_Canton,customer_city_Canyon Country,customer_city_Carlisle,customer_city_Carlsbad,customer_city_Carmichael,customer_city_Carol Stream,customer_city_Carolina,customer_city_Carrollton,customer_city_Carson,customer_city_Cary,customer_city_Catonsville,customer_city_Cayey,customer_city_Cerritos,customer_city_Chambersburg,customer_city_Chandler,customer_city_Chapel Hill,customer_city_Charlotte,customer_city_Chesapeake,customer_city_Chicago,customer_city_Chicago Heights,customer_city_Chillicothe,customer_city_Chino,customer_city_Chino Hills,customer_city_Chula Vista,customer_city_Cicero,customer_city_Cincinnati,customer_city_Citrus Heights,customer_city_Clarksville,customer_city_Clearfield,customer_city_Clementon,customer_city_Cleveland,customer_city_Clovis,customer_city_College Station,customer_city_Colorado Springs,customer_city_Colton,customer_city_Columbia,customer_city_Columbus,customer_city_Compton,customer_city_Conway,customer_city_Cordova,customer_city_Corona,customer_city_Costa Mesa,customer_city_Crown Point,customer_city_Crystal Lake,customer_city_Cumberland,customer_city_Cupertino,customer_city_Cypress,customer_city_Dallas,customer_city_Daly City,customer_city_Danbury,customer_city_Davis,customer_city_Dayton,customer_city_Dearborn,customer_city_Decatur,customer_city_Del Rio,customer_city_Denton,customer_city_Denver,c

### 3.3.3 One Hot Encode Categorical Columns

In [17]:
def one_hot_encode(df_data: pd.DataFrame) -> pd.DataFrame:
    """
    Perform one-hot encoding on the categorical columns 
    in the dataframe.

    Parameters
    ----------
    df_data : pd.DataFrame
        The source dataframe

    Returns
    -------
    pd.DataFrame
        The dataframe with the columns one-hot encoded.
    """
    df_output = df_data.copy()

    # one-hot encode the categorical columns
    for column in df_output.select_dtypes(include='category').columns:
        df_output = pd.concat([df_output, pd.get_dummies(df_output[column], prefix=column)], axis=1)
        df_output.drop(columns=[column], inplace=True)
        
    return df_output
    
df_features = one_hot_encode(df_features)

print(df_features.shape)
with pd.option_context('display.max_columns', None):
    display(df_features.head())

(65752, 818)


,is_fraud,order_total,sales_per_customer,payment_type_CASH,payment_type_DEBIT,payment_type_PAYMENT,payment_type_TRANSFER,customer_country_EE. UU.,customer_country_Puerto Rico,customer_state_91732,customer_state_95758,customer_state_AL,customer_state_AR,customer_state_AZ,customer_state_CA,customer_state_CO,customer_state_CT,customer_state_DC,customer_state_DE,customer_state_FL,customer_state_GA,customer_state_HI,customer_state_IA,customer_state_ID,customer_state_IL,customer_state_IN,customer_state_KS,customer_state_KY,customer_state_LA,customer_state_MA,customer_state_MD,customer_state_MI,customer_state_MN,customer_state_MO,customer_state_MT,customer_state_NC,customer_state_ND,customer_state_NJ,customer_state_NM,customer_state_NV,customer_state_NY,customer_state_OH,customer_state_OK,customer_state_OR,customer_state_PA,customer_state_PR,customer_state_RI,customer_state_SC,customer_state_TN,customer_state_TX,customer_state_UT,customer_state_VA,customer_state_WA,customer_state_WI,customer_state_WV,customer_city_Aguadilla,customer_city_Alameda,customer_city_Albany,customer_city_Albuquerque,customer_city_Algonquin,customer_city_Alhambra,customer_city_Allentown,customer_city_Alpharetta,customer_city_Amarillo,customer_city_Anaheim,customer_city_Ann Arbor,customer_city_Annandale,customer_city_Annapolis,customer_city_Antioch,customer_city_Apex,customer_city_Apopka,customer_city_Arecibo,customer_city_Arlington,customer_city_Arlington Heights,customer_city_Asheboro,customer_city_Astoria,customer_city_Atlanta,customer_city_Augusta,customer_city_Aurora,customer_city_Austin,customer_city_Azusa,customer_city_Bakersfield,customer_city_Baldwin Park,customer_city_Ballwin,customer_city_Baltimore,customer_city_Bartlett,customer_city_Bay Shore,customer_city_Bayamon,customer_city_Bayonne,customer_city_Baytown,customer_city_Beaverton,customer_city_Bell Gardens,customer_city_Bellflower,customer_city_Bellingham,customer_city_Beloit,customer_city_Bend,customer_city_Bensalem,customer_city_Berwyn,customer_city_Billings,customer_city_Birmingham,customer_city_Bismarck,customer_city_Blacksburg,customer_city_Bloomfield,customer_city_Bolingbrook,customer_city_Bountiful,customer_city_Bowling Green,customer_city_Brandon,customer_city_Brentwood,customer_city_Bridgeton,customer_city_Brighton,customer_city_Bristol,customer_city_Brockton,customer_city_Broken Arrow,customer_city_Bronx,customer_city_Brooklyn,customer_city_Broomfield,customer_city_Brownsville,customer_city_Buena Park,customer_city_Buffalo,customer_city_Burnsville,customer_city_CA,customer_city_Caguas,customer_city_Campbell,customer_city_Canoga Park,customer_city_Canovanas,customer_city_Canton,customer_city_Canyon Country,customer_city_Carlisle,customer_city_Carlsbad,customer_city_Carmichael,customer_city_Carol Stream,customer_city_Carolina,customer_city_Carrollton,customer_city_Carson,customer_city_Cary,customer_city_Catonsville,customer_city_Cayey,customer_city_Cerritos,customer_city_Chambersburg,customer_city_Chandler,customer_city_Chapel Hill,customer_city_Charlotte,customer_city_Chesapeake,customer_city_Chicago,customer_city_Chicago Heights,customer_city_Chillicothe,customer_city_Chino,customer_city_Chino Hills,customer_city_Chula Vista,customer_city_Cicero,customer_city_Cincinnati,customer_city_Citrus Heights,customer_city_Clarksville,customer_city_Clearfield,customer_city_Clementon,customer_city_Cleveland,customer_city_Clovis,customer_city_College Station,customer_city_Colorado Springs,customer_city_Colton,customer_city_Columbia,customer_city_Columbus,customer_city_Compton,customer_city_Conway,customer_city_Cordova,customer_city_Corona,customer_city_Costa Mesa,customer_city_Crown Point,customer_city_Crystal Lake,customer_city_Cumberland,customer_city_Cupertino,customer_city_Cypress,customer_city_Dallas,customer_city_Daly City,customer_city_Danbury,customer_city_Davis,customer_city_Dayton,customer_city_Dearborn,customer_city_Decatur,customer_city_Del Rio,customer_city_Denton,customer_city_Denver,c